In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [73]:
driver = webdriver.Chrome('/Users/wenyunie/Downloads/chromedriver')

In [74]:
driver.get('http://www.51job.com')

In [75]:
element = driver.find_element_by_id('kwdselectid1')
action = webdriver.common.action_chains.ActionChains(driver)
action.move_to_element_with_offset(element,5,5)
time.sleep(3)
action.click()
time.sleep(3)
action.send_keys('aaa')
time.sleep(3)
action.perform()

In [33]:
# 模拟回车操作 ,开始搜索
driver.find_element_by_id('kwdselectid1').clear()
time.sleep(3)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)


In [157]:
requests.adapters.DEFAULT_RETRIES = 5 # 增加重连次数

In [164]:
def get_page(url):
    headers = {
    'Host': 'search.51job.com',
    'Referer': url,
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    job_ids = []
    job_names = []
    job_links = []
    company_names = []
    company_links = []
    
    s = requests.session()
    s.keep_alive = False # 关闭多余连接
    r = s.get(url,headers=headers)
    
    soup = BeautifulSoup(r.content,'html.parser',from_encoding="gb18030")

    # job id
    for item in soup.find_all('input','checkbox'):
        job_ids.append(item['value'])

    # job name
    for item in soup.find_all('a',target=True,onmousedown=True):
        if item.img is None:
            job_names.append(item['title'])

    # company name
    for company in soup.find_all('span','t2'):
        if company.a is not None:
            company_names.append(company.get_text())
            company_links.append(company.a['href'])

    # job link
    row_list = []
    for item in soup.find_all('p','t1'):
        row_list.append(get_job(item.a['href']))
        job_links.append(item.a['href'])
    search_page = pd.DataFrame(row_list)
    search_page['job_id'] = job_ids
    search_page['job_name'] = job_names
    search_page['company_name'] = company_names
    search_page['company_link'] = company_links
    search_page['job_link'] = job_links
    
    return(search_page)

In [165]:
def get_job(url):
    headers = {
        'Host': 'jobs.51job.com',
        'Referer': url,
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }
    
    s = requests.session()
    s.keep_alive = False # close the unnecessary connections
    r = s.get(url,headers=headers)
    
    soup = BeautifulSoup(r.content,'html.parser',from_encoding="gb18030")
    salary = None
    job_info_basic = None
    job_info_detail = None
    job_type_tags = None
    job_keyword_tags = None
    
    if soup.find('div','tHeader tHjob') is not None:
        # salary
        salary = soup.find('div','tHeader tHjob').strong.get_text()

        # basic job info
        job_info_basic = soup.find('div','tHeader tHjob').find('p','msg ltype').get_text()

        # detail job info
        job_info_detail = soup.find('div','bmsg job_msg inbox').get_text()

        # job tags
        content = soup.find('div','mt10').find_all('p')
        job_type_tags = content[0].get_text() # job type tags
        job_keyword_tags = None
        if len(content)>1:
            job_keyword_tags = content[1].get_text() # job keyword tags
    return(
    {
        'salary':salary,
        'job_info_basic':job_info_basic,
        'job_info_detail':job_info_detail,
        'job_type':job_type_tags,
        'job_keyword':job_keyword_tags
    })

In [166]:
def get_market_51job(url,n_page):
    for i in range(n_page):
        url = url.format(i+1)
        if i == 0:
            job_market_info = get_page(url)
        job_market_info = pd.concat([job_market_info,get_page(url)])
        print("{0}% of the data on the website has been scraped down!".format(round((i+1)/n_page*100)))
    return(job_market_info)

In [171]:
url = 'https://search.51job.com/list/020000,000000,0000,00,9,99,%25E5%25BF%2583%25E7%2590%2586%25E5%25AD%25A6,2,{0}.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare='
psych = get_market_51job(url,112)

1% of the data on the website has been scraped down!
2% of the data on the website has been scraped down!
3% of the data on the website has been scraped down!
4% of the data on the website has been scraped down!
4% of the data on the website has been scraped down!
5% of the data on the website has been scraped down!
6% of the data on the website has been scraped down!
7% of the data on the website has been scraped down!
8% of the data on the website has been scraped down!
9% of the data on the website has been scraped down!
10% of the data on the website has been scraped down!
11% of the data on the website has been scraped down!
12% of the data on the website has been scraped down!
12% of the data on the website has been scraped down!
13% of the data on the website has been scraped down!
14% of the data on the website has been scraped down!
15% of the data on the website has been scraped down!
16% of the data on the website has been scraped down!
17% of the data on the website has bee

In [203]:
# psych.to_csv('psych_market_51job.csv')
psych['job_info_basic'] = psych['job_info_basic'].str.slice(start = 6).str.split('|')
location = []
experience = []
education = []
demand = []
psych['job_info_basic'].size
for info in psych['job_info_basic']:
    if info is not None:
        location.append(info[0])
        if len(info) == 5:
            experience.append(info[1])
            education.append(info[2])
            demand.append(info[3])
        if len(info) == 4:
            experience.append(info[1])
            education.append(None)
            demand.append(info[2])
    else:
        location.append(None)
        experience.append(None)
        education.append(None)
        demand.append(None)

TypeError: 'float' object is not subscriptable

In [215]:
psych_market_51job['job_info_basic']

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
      ..
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
Name: job_info_basic, Length: 5650, dtype: float64

In [202]:
psych['location'] = location
psych['experience'] = experience
psych['education'] = education
psych['demand'] = demand

ValueError: Length of values does not match length of index